In [1]:
!pip install pyspark
!pip install findspark
!pip install pandas 
!pip install install-jdk

In [2]:
import pandas as pd
import findspark
findspark.init()

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.master('local[*]').getOrCreate()

23/06/17 10:27:20 WARN Utils: Your hostname, danilojpfreitas-Vostro-13-5310 resolves to a loopback address: 127.0.1.1; using 10.0.0.155 instead (on interface wlp0s20f3)
23/06/17 10:27:20 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/17 10:27:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
spark

# Carregamendo dos Dados

## Distância entre Maceió e outras capitais

In [6]:
distPath = '/home/danilojpfreitas/Documentos/Data Engineer/Projeto/DataPipeline-TarifasAereas/data/dist/maceioDistCapitals.json'

In [7]:
dist = spark.read.option("multiline", "true").json(distPath)

In [8]:
dist.show()

+--------------+-----------+--------+
|          city|distHighway|distLine|
+--------------+-----------+--------+
|    Rio Branco|       5063|    3513|
|        Macapa|       3545|    2011|
|        Manaus|       5444|    2779|
|      Salvador|        578|     476|
|     Fortaleza|        965|     731|
|      Brasilia|       1920|    1487|
|       Vitoria|       1611|    1282|
|       Goiania|       2119|    1658|
|      Sao Luis|       1584|    1236|
|        Cuiaba|       3111|    2304|
|  Campo Grande|       2830|    2355|
|Belo Horizonte|       1855|    1443|
|         Belem|       2059|    1682|
|   Joao Pessoa|        371|     300|
|      Curitiba|       2805|    2263|
|        Recife|        257|     203|
|      Teresina|       1155|     931|
|      Teresina|       2071|    1674|
|         Natal|        539|     435|
|  Porto Alegre|       3531|    2778|
+--------------+-----------+--------+
only showing top 20 rows



## Preço por ano das passagens

### Carregamento dos dados

In [294]:
price2019Path = "../data/price2019/*.CSV"
price2020Path = "../data/price2020/*.CSV"
price2021Path = "../data/price2021/*.CSV"
price2022Path = "../data/price2022/*.CSV"

In [295]:
price2019 = spark.read.csv(price2019Path, sep=';', inferSchema=True)
price2020 = spark.read.csv(price2020Path, sep=';', inferSchema=True)
price2021 = spark.read.csv(price2021Path, sep=';', inferSchema=True)
price2022 = spark.read.csv(price2022Path, sep=';', inferSchema=True)

In [296]:
price2019.show()

+----+---+-------+------+-------+------+--------+
| _c0|_c1|    _c2|   _c3|    _c4|   _c5|     _c6|
+----+---+-------+------+-------+------+--------+
| ANO|MES|EMPRESA|ORIGEM|DESTINO|TARIFA|ASSENTOS|
|2019|  1|    AZU|  SBAC|   SBAR|239,90|       2|
|2019|  1|    AZU|  SBAC|   SBAR|270,90|       2|
|2019|  1|    AZU|  SBAC|   SBBR|217,90|       1|
|2019|  1|    AZU|  SBAC|   SBBR|630,90|       2|
|2019|  1|    AZU|  SBAC|   SBBV|347,90|       1|
|2019|  1|    AZU|  SBAC|   SBCF|212,90|       2|
|2019|  1|    AZU|  SBAC|   SBCF|283,90|       1|
|2019|  1|    AZU|  SBAC|   SBCF|354,90|       3|
|2019|  1|    AZU|  SBAC|   SBCF|414,90|       2|
|2019|  1|    AZU|  SBAC|   SBCF|619,90|       3|
|2019|  1|    AZU|  SBAC|   SBCG|291,90|       1|
|2019|  1|    AZU|  SBAC|   SBCH|737,90|       2|
|2019|  1|    AZU|  SBAC|   SBCR|361,90|       1|
|2019|  1|    AZU|  SBAC|   SBCY|425,90|       1|
|2019|  1|    AZU|  SBAC|   SBEG|277,90|       1|
|2019|  1|    AZU|  SBAC|   SBEG|307,90|       2|


### Renomeando o Header

In [297]:
newHeaderColNames = ['year', 'month', 'company', 'origin', 'destination', 'price', 'seats']

In [298]:
for index, colName in enumerate(newHeaderColNames):
    price2019 = price2019.withColumnRenamed(f"_c{index}", colName)
    price2020 = price2020.withColumnRenamed(f"_c{index}", colName)
    price2021 = price2021.withColumnRenamed(f"_c{index}", colName)
    price2022 = price2022.withColumnRenamed(f"_c{index}", colName)

In [299]:
price2020.limit(5).show()

+-----------------+-----------------+------------------+--------------------+--------------------+--------+--------------------+
|             year|            month|           company|              origin|         destination|   price|               seats|
+-----------------+-----------------+------------------+--------------------+--------------------+--------+--------------------+
|Ano de Refer�ncia|M�s de Refer�ncia|ICAO Empresa A�rea|ICAO Aer�dromo Or...|ICAO Aer�dromo De...|Tarifa-N|Assentos Comercia...|
|             2020|               11|               AZU|                SBAC|                SBBR|  297,69|                   5|
|             2020|               11|               AZU|                SBAC|                SBBR|   340,9|                   1|
|             2020|               11|               AZU|                SBAC|                SBBR|   374,9|                   3|
|             2020|               11|               AZU|                SBAC|                SBBV

In [300]:
price2019 = price2019.where('price!="Tarifa-N"')
price2020 = price2020.where('price!="Tarifa-N"')
price2021 = price2021.where('price!="Tarifa-N"')
price2022 = price2022.where('price!="Tarifa-N"')

In [301]:
price2022.limit(5).show()

+----+-----------------+-----------------+------------------+--------------------+--------------------+--------+--------------------+
|year|            month|          company|            origin|         destination|               price|   seats|                 _c7|
+----+-----------------+-----------------+------------------+--------------------+--------------------+--------+--------------------+
|null|Ano de Refer�ncia|M�s de Refer�ncia|ICAO Empresa A�rea|ICAO Aer�dromo Or...|ICAO Aer�dromo De...|Tarifa-N|Assentos Comercia...|
|   1|             2022|                4|               AZU|                SBQV|                SBBE|  3500,9|                   1|
|   2|             2022|                4|               AZU|                SBCA|                SBCT|   814,9|                   5|
|   3|             2022|                4|               AZU|                SBCF|                SBGO|   814,9|                   6|
|   4|             2022|                4|               AZU| 

### Selecionamento somente o aeroporto de Maceió como origem

In [302]:
price2019 = price2019.where('origin=="SBMO"')
price2020 = price2020.where('origin=="SBMO"')
price2021 = price2021.where('origin=="SBMO"')
price2022 = price2022.where('origin=="SBMO"')

In [303]:
price2020.limit(5).show()

+----+-----+-------+------+-----------+------+-----+
|year|month|company|origin|destination| price|seats|
+----+-----+-------+------+-----------+------+-----+
|2020|   11|    AZU|  SBMO|       SBAE|1370,9|    2|
|2020|   11|    AZU|  SBMO|       SBAE|1409,9|    2|
|2020|   11|    AZU|  SBMO|       SBAE| 352,8|    6|
|2020|   11|    AZU|  SBMO|       SBAE|378,05|    1|
|2020|   11|    AZU|  SBMO|       SBAE| 415,9|    2|
+----+-----+-------+------+-----------+------+-----+



In [304]:
price2019.count()

62530

### Convertendo: String to Integer and String to Double

In [20]:
from pyspark.sql.types import DoubleType, StringType, IntegerType 

In [21]:
from pyspark.sql import functions as f

#### 2019

In [22]:
price2019.printSchema()

root
 |-- year: string (nullable = true)
 |-- month: string (nullable = true)
 |-- company: string (nullable = true)
 |-- origin: string (nullable = true)
 |-- destination: string (nullable = true)
 |-- price: string (nullable = true)
 |-- seats: string (nullable = true)



In [23]:
price2019 = price2019\
    .withColumn(
    "year",
    price2019["year"].cast(IntegerType())
    )\
    .withColumn(
    "month",
    price2019["month"].cast(IntegerType())
    )\
    .withColumn(
    "seats",
    price2019["seats"].cast(IntegerType())
    )\
    .withColumn(
    "price",
    f.regexp_replace('price', ',', '.')
    )

In [24]:
price2019 = price2019\
    .withColumn(
    "price",
    price2019["price"].cast(DoubleType())
    )

In [25]:
price2019.printSchema()

root
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- company: string (nullable = true)
 |-- origin: string (nullable = true)
 |-- destination: string (nullable = true)
 |-- price: double (nullable = true)
 |-- seats: integer (nullable = true)



In [26]:
price2019.limit(10).show()

+----+-----+-------+------+-----------+------+-----+
|year|month|company|origin|destination| price|seats|
+----+-----+-------+------+-----------+------+-----+
|2019|    1|    AZU|  SBMO|       SBAC| 179.9|    2|
|2019|    1|    AZU|  SBMO|       SBAE|295.33|    2|
|2019|    1|    AZU|  SBMO|       SBAE| 302.9|    2|
|2019|    1|    AZU|  SBMO|       SBAE| 372.9|    2|
|2019|    1|    AZU|  SBMO|       SBAE|389.42|    4|
|2019|    1|    AZU|  SBMO|       SBAE| 419.9|    2|
|2019|    1|    AZU|  SBMO|       SBAE| 448.9|    1|
|2019|    1|    AZU|  SBMO|       SBAE| 639.9|    1|
|2019|    1|    AZU|  SBMO|       SBAE| 656.9|    1|
|2019|    1|    AZU|  SBMO|       SBAE| 724.9|    1|
+----+-----+-------+------+-----------+------+-----+



#### 2020

In [27]:
price2020 = price2020\
    .withColumn(
    "year",
    price2020["year"].cast(IntegerType())
    )\
    .withColumn(
    "month",
    price2020["month"].cast(IntegerType())
    )\
    .withColumn(
    "seats",
    price2020["seats"].cast(IntegerType())
    )\
    .withColumn(
    "price",
    f.regexp_replace('price', ',', '.')
    )

In [28]:
price2020 = price2020\
    .withColumn(
    "price",
    price2020["price"].cast(DoubleType())
    )

In [29]:
price2020.printSchema()

root
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- company: string (nullable = true)
 |-- origin: string (nullable = true)
 |-- destination: string (nullable = true)
 |-- price: double (nullable = true)
 |-- seats: integer (nullable = true)



#### 2021

In [30]:
price2021 = price2021\
    .withColumn(
    "year",
    price2021["year"].cast(IntegerType())
    )\
    .withColumn(
    "month",
    price2021["month"].cast(IntegerType())
    )\
    .withColumn(
    "seats",
    price2021["seats"].cast(IntegerType())
    )\
    .withColumn(
    "price",
    f.regexp_replace('price', ',', '.')
    )

In [31]:
price2021 = price2021\
    .withColumn(
    "price",
    price2021["price"].cast(DoubleType())
    )

In [32]:
price2021.printSchema()

root
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- company: string (nullable = true)
 |-- origin: string (nullable = true)
 |-- destination: string (nullable = true)
 |-- price: double (nullable = true)
 |-- seats: integer (nullable = true)
 |-- _c7: string (nullable = true)



#### 2022

In [305]:
price2022 = price2022\
    .withColumn(
    "year",
    price2022["year"].cast(IntegerType())
    )\
    .withColumn(
    "month",
    price2022["month"].cast(IntegerType())
    )\
    .withColumn(
    "seats",
    price2022["seats"].cast(IntegerType())
    )\
    .withColumn(
    "price",
    f.regexp_replace('price', ',', '.')
    )

In [306]:
price2022 = price2022\
    .withColumn(
    "price",
    price2022["price"].cast(DoubleType())
    )

In [307]:
price2022.printSchema()

root
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- company: string (nullable = true)
 |-- origin: string (nullable = true)
 |-- destination: string (nullable = true)
 |-- price: double (nullable = true)
 |-- seats: integer (nullable = true)
 |-- _c7: string (nullable = true)



In [308]:
price2022.limit(5).show()

+----+-----+-------+------+-----------+------+-----+----+
|year|month|company|origin|destination| price|seats| _c7|
+----+-----+-------+------+-----------+------+-----+----+
|2022|   10|    AZU|  SBMO|       SBAE|1295.9|    3|null|
|2022|   10|    AZU|  SBMO|       SBAE|1878.9|    1|null|
|2022|   10|    AZU|  SBMO|       SBAE|1973.9|    1|null|
|2022|   10|    AZU|  SBMO|       SBAE|2023.9|    1|null|
|2022|   10|    AZU|  SBMO|       SBAE|2475.9|    1|null|
+----+-----+-------+------+-----------+------+-----+----+



## Codigo ICAO - JSON

###  ICAO da Base de Dados faltando Aeroportos => Utilização de uma nova base de dados

In [36]:
pathIcaoJson = '/home/danilojpfreitas/Documentos/Data Engineer/Projeto/DataPipeline-TarifasAereas/data/icao/icao.csv'

In [37]:
icao = spark.read.csv(pathIcaoJson, sep=',', header=True)

In [38]:
icao.printSchema()

root
 |-- icao: string (nullable = true)
 |-- airport: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)



In [39]:
icao.limit(5).show()

+----+--------------------+----------------+-----+
|icao|             airport|            city|state|
+----+--------------------+----------------+-----+
|SBFE|JOÃO DURVAL CARNEIRO|FEIRA DE SANTANA|   BA|
|SSPB|         PATO BRANCO|     PATO BRANCO|   PR|
|SBRG|          RIO GRANDE|      RIO GRANDE|   RS|
|SBGU|TANCREDO THOMAS D...|      GUARAPUAVA|   PR|
|SSOU|            ARIPUANÃ|        ARIPUANÃ|   MT|
+----+--------------------+----------------+-----+



In [40]:
icao.select("icao", "city", "state").limit(5).show()

+----+----------------+-----+
|icao|            city|state|
+----+----------------+-----+
|SBFE|FEIRA DE SANTANA|   BA|
|SSPB|     PATO BRANCO|   PR|
|SBRG|      RIO GRANDE|   RS|
|SBGU|      GUARAPUAVA|   PR|
|SSOU|        ARIPUANÃ|   MT|
+----+----------------+-----+



In [41]:
icao.where('icao==""').show()

+----+-------+----+-----+
|icao|airport|city|state|
+----+-------+----+-----+
+----+-------+----+-----+



## Dados Estatisticos

In [42]:
pathEstatisticos = '/home/danilojpfreitas/Documentos/Data Engineer/Projeto/DataPipeline-TarifasAereas/data/statistic/Dados_Estatisticos.csv'

In [43]:
estatisticos = spark.read.csv(pathEstatisticos, sep=';', header=True)

In [44]:
estatisticos.printSchema()

root
 |-- EMPRESA_SIGLA: string (nullable = true)
 |-- EMPRESA_NOME: string (nullable = true)
 |-- EMPRESA_NACIONALIDADE: string (nullable = true)
 |-- ANO: string (nullable = true)
 |-- MES: string (nullable = true)
 |-- AEROPORTO_DE_ORIGEM_SIGLA: string (nullable = true)
 |-- AEROPORTO_DE_ORIGEM_NOME: string (nullable = true)
 |-- AEROPORTO_DE_ORIGEM_UF: string (nullable = true)
 |-- AEROPORTO_DE_ORIGEM_REGIAO: string (nullable = true)
 |-- AEROPORTO_DE_ORIGEM_PAIS: string (nullable = true)
 |-- AEROPORTO_DE_ORIGEM_CONTINENTE: string (nullable = true)
 |-- AEROPORTO_DE_DESTINO_SIGLA: string (nullable = true)
 |-- AEROPORTO_DE_DESTINO_NOME: string (nullable = true)
 |-- AEROPORTO_DE_DESTINO_UF: string (nullable = true)
 |-- AEROPORTO_DE_DESTINO_REGIAO: string (nullable = true)
 |-- AEROPORTO_DE_DESTINO_PAIS: string (nullable = true)
 |-- AEROPORTO_DE_DESTINO_CONTINENTE: string (nullable = true)
 |-- NATUREZA: string (nullable = true)
 |-- GRUPO_DE_VOO: string (nullable = true)
 |-- PA

In [45]:
estatisticos.createOrReplaceTempView('estatisticosSQLView')

23/06/17 10:28:02 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [46]:
estatisticosSelect = spark.sql("""
        SELECT
        EMPRESA_SIGLA AS company,
        EMPRESA_NOME AS companyName,
        EMPRESA_NACIONALIDADE AS nationality,
        ANO AS year,
        MES AS month,
        AEROPORTO_DE_ORIGEM_SIGLA AS origin,
        AEROPORTO_DE_DESTINO_SIGLA AS destination,
        DECOLAGENS AS takeOffs,
        DISTANCIA_VOADA_KM AS distance,
        ASSENTOS AS seats,
        HORAS_VOADAS AS time,
        PASSAGEIROS_PAGOS AS paidPassengers,
        PASSAGEIROS_GRATIS AS noPaidPassengers
        FROM
        estatisticosSQLView
""")

In [47]:
estatisticosSelect.limit(5).show()

+-------+--------------------+-----------+----+-----+------+-----------+--------+--------+-----+------+--------------+----------------+
|company|         companyName|nationality|year|month|origin|destination|takeOffs|distance|seats|  time|paidPassengers|noPaidPassengers|
+-------+--------------------+-----------+----+-----+------+-----------+--------+--------+-----+------+--------------+----------------+
|    AAL|AMERICAN AIRLINES...|ESTRANGEIRA|2000|    1|  KDFW|       SBGR|      30|  247320| 6330|409,08|          3932|             142|
|    AAL|AMERICAN AIRLINES...|ESTRANGEIRA|2000|    1|  KJFK|       SBGL|      29|  224141| 6119| 418,1|          2338|             131|
|    AAL|AMERICAN AIRLINES...|ESTRANGEIRA|2000|    1|  KJFK|       SBGR|      29|  222256| 6119| 411,5|          2892|             127|
|    AAL|AMERICAN AIRLINES...|ESTRANGEIRA|2000|    1|  KMIA|       SBCF|    null|    null| null|  null|           997|              20|
|    AAL|AMERICAN AIRLINES...|ESTRANGEIRA|2000| 

In [48]:
estatisticosSelect = estatisticosSelect\
    .withColumn(
    "year",
    estatisticosSelect["year"].cast(IntegerType())
    )\
    .withColumn(
    "month",
    estatisticosSelect["month"].cast(IntegerType())
    )\
    .withColumn(
    "takeOffs",
    estatisticosSelect["takeOffs"].cast(IntegerType())
    )\
    .withColumn(
    "distance",
    estatisticosSelect["distance"].cast(IntegerType())
    )\
    .withColumn(
    "seats",
    estatisticosSelect["seats"].cast(IntegerType())
    )\
    .withColumn(
    "time",
    f.regexp_replace('time', ',', '.')
    )\
    .withColumn(
    "paidPassengers",
    estatisticosSelect["paidPassengers"].cast(IntegerType())
    )\
    .withColumn(
    "noPaidPassengers",
    estatisticosSelect["noPaidPassengers"].cast(IntegerType())
    )

In [49]:
estatisticosSelect.printSchema()

root
 |-- company: string (nullable = true)
 |-- companyName: string (nullable = true)
 |-- nationality: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- origin: string (nullable = true)
 |-- destination: string (nullable = true)
 |-- takeOffs: integer (nullable = true)
 |-- distance: integer (nullable = true)
 |-- seats: integer (nullable = true)
 |-- time: string (nullable = true)
 |-- paidPassengers: integer (nullable = true)
 |-- noPaidPassengers: integer (nullable = true)



# Carregamento na Processing Zone - Arquivos no Formato Parquet

## Preço por ano das passagens

#### 2019

In [50]:
pathProcessingZone2019 = "/home/danilojpfreitas/Documentos/Data Engineer/Projeto/DataPipeline-TarifasAereas/data/processing/2019"

In [51]:
price2019.write.parquet(
    pathProcessingZone2019,
    mode = 'overwrite'
)

23/06/17 10:28:09 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers


In [52]:
price2019Parquet = spark.read.parquet(
    pathProcessingZone2019
)

In [53]:
price2019Parquet.limit(5).show()

+----+-----+-------+------+-----------+-----+-----+
|year|month|company|origin|destination|price|seats|
+----+-----+-------+------+-----------+-----+-----+
|2019|    5|    AZU|  SBMO|       SBAE|558.9|    3|
|2019|    5|    AZU|  SBMO|       SBAE|633.9|    3|
|2019|    5|    AZU|  SBMO|       SBAE|648.9|    1|
|2019|    5|    AZU|  SBMO|       SBAE|723.9|    2|
|2019|    5|    AZU|  SBMO|       SBAE|773.9|    2|
+----+-----+-------+------+-----------+-----+-----+



In [54]:
price2019Parquet.count()

62530

#### 2020

In [200]:
pathProcessingZone2020 = "/home/danilojpfreitas/Documentos/Data Engineer/Projeto/DataPipeline-TarifasAereas/data/processing/2020"

In [201]:
price2020.write.parquet(
    pathProcessingZone2020,
    mode = 'overwrite'
)

23/06/17 15:27:52 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers


In [202]:
price2020Parquet = spark.read.parquet(
    pathProcessingZone2020
)

In [203]:
price2020Parquet.limit(5).show()

+----+-----+-------+------+-----------+------+-----+
|year|month|company|origin|destination| price|seats|
+----+-----+-------+------+-----------+------+-----+
|2020|   11|    AZU|  SBMO|       SBAE|1370.9|    2|
|2020|   11|    AZU|  SBMO|       SBAE|1409.9|    2|
|2020|   11|    AZU|  SBMO|       SBAE| 352.8|    6|
|2020|   11|    AZU|  SBMO|       SBAE|378.05|    1|
|2020|   11|    AZU|  SBMO|       SBAE| 415.9|    2|
+----+-----+-------+------+-----------+------+-----+



#### 2021

In [59]:
pathProcessingZone2021 = "/home/danilojpfreitas/Documentos/Data Engineer/Projeto/DataPipeline-TarifasAereas/data/processing/2021"

In [60]:
price2021.write.parquet(
    pathProcessingZone2021,
    mode = 'overwrite'
)

23/06/17 10:28:17 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers


In [61]:
price2021Parquet = spark.read.parquet(
    pathProcessingZone2021
)

In [62]:
price2021Parquet.limit(5).show()

+----+-----+-------+------+-----------+-----+-----+----+
|year|month|company|origin|destination|price|seats| _c7|
+----+-----+-------+------+-----------+-----+-----+----+
|2021|   11|    AZU|  SBMO|       SBAE|538.9|    2|null|
|2021|   11|    AZU|  SBMO|       SBAE|557.9|    1|null|
|2021|   11|    AZU|  SBMO|       SBAE|592.9|    1|null|
|2021|   11|    AZU|  SBMO|       SBAE|697.9|    1|null|
|2021|   11|    AZU|  SBMO|       SBAE|698.9|    1|null|
+----+-----+-------+------+-----------+-----+-----+----+



#### 2022

In [309]:
pathProcessingZone2022 = "/home/danilojpfreitas/Documentos/Data Engineer/Projeto/DataPipeline-TarifasAereas/data/processing/2022"

In [310]:
price2022.write.parquet(
    pathProcessingZone2022,
    mode = 'overwrite'
)

23/06/17 15:42:08 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers


In [311]:
price2022Parquet = spark.read.parquet(
    pathProcessingZone2022
)

In [312]:
price2022Parquet.limit(5).show()

+----+-----+-------+------+-----------+------+-----+----+
|year|month|company|origin|destination| price|seats| _c7|
+----+-----+-------+------+-----------+------+-----+----+
|2022|   12|    AZU|  SBMO|       SBAE| 661.9|    1|null|
|2022|   12|    AZU|  SBMO|       SBAE| 756.9|    2|null|
|2022|   12|    AZU|  SBMO|       SBAE|842.89|    1|null|
|2022|   12|    AZU|  SBMO|       SBAE| 869.9|    1|null|
|2022|   12|    AZU|  SBMO|       SBAE| 927.9|    2|null|
+----+-----+-------+------+-----------+------+-----+----+



## Distância entre Maceió e outras capitais

In [67]:
distParquet = "/home/danilojpfreitas/Documentos/Data Engineer/Projeto/DataPipeline-TarifasAereas/data/processing/distancia"

In [68]:
dist.write.parquet(
    distParquet,
    mode = 'overwrite'
)

In [69]:
dist_parquet = spark.read.parquet(distParquet)

In [70]:
dist_parquet.show()

+--------------+-----------+--------+
|          city|distHighway|distLine|
+--------------+-----------+--------+
|    Rio Branco|       5063|    3513|
|        Macapa|       3545|    2011|
|        Manaus|       5444|    2779|
|      Salvador|        578|     476|
|     Fortaleza|        965|     731|
|      Brasilia|       1920|    1487|
|       Vitoria|       1611|    1282|
|       Goiania|       2119|    1658|
|      Sao Luis|       1584|    1236|
|        Cuiaba|       3111|    2304|
|  Campo Grande|       2830|    2355|
|Belo Horizonte|       1855|    1443|
|         Belem|       2059|    1682|
|   Joao Pessoa|        371|     300|
|      Curitiba|       2805|    2263|
|        Recife|        257|     203|
|      Teresina|       1155|     931|
|      Teresina|       2071|    1674|
|         Natal|        539|     435|
|  Porto Alegre|       3531|    2778|
+--------------+-----------+--------+
only showing top 20 rows



## Codigo ICAO - JSON

In [71]:
icaoParquet = "/home/danilojpfreitas/Documentos/Data Engineer/Projeto/DataPipeline-TarifasAereas/data/processing/icao"

In [72]:
icao.write.parquet(
    icaoParquet,
    mode = 'overwrite'
)

In [73]:
icao_parquet = spark.read.parquet(
    icaoParquet
)

In [74]:
icao_parquet.show()

+----+--------------------+--------------------+-----+
|icao|             airport|                city|state|
+----+--------------------+--------------------+-----+
|SBFE|JOÃO DURVAL CARNEIRO|    FEIRA DE SANTANA|   BA|
|SSPB|         PATO BRANCO|         PATO BRANCO|   PR|
|SBRG|          RIO GRANDE|          RIO GRANDE|   RS|
|SBGU|TANCREDO THOMAS D...|          GUARAPUAVA|   PR|
|SSOU|            ARIPUANÃ|            ARIPUANÃ|   MT|
|SBSI|   FAZENDA ITAGUASSU|            MARACAJU|   MS|
|SWSI|PRESIDENTE JOÃO B...|              SINOPE|   MG|
|SBQV|PEDRO OTACÍLIO FI...|VITÓRIA DA CONQUISTA|   BA|
|SBAA|CONCEIÇÃO DO ARAG...|CONCEIÇÃO DO ARAG...|   PA|
|SBAE|       BAURU/AREALVA|               BAURU|   SP|
|SBAQ|BARTOLOMEU DE GUSMÃO|          ARARAQUARA|   SP|
|SBAR|         SANTA MARIA|             ARACAJU|   SE|
|SBAT|PILOTO OSVALDO MA...|       ALTA FLORESTA|   MT|
|SBAU|           ARAÇATUBA|           ARAÇATUBA|   SP|
|SBAX|          ROMEU ZEMA|               ARAXÁ|   MG|
|SBBE|INTE

## Dados Estatisticos

In [75]:
estatisticosSelect.count()

969277

In [76]:
estatisticosSelect.show()

+-------+--------------------+-----------+----+-----+------+-----------+--------+--------+-----+-------+--------------+----------------+
|company|         companyName|nationality|year|month|origin|destination|takeOffs|distance|seats|   time|paidPassengers|noPaidPassengers|
+-------+--------------------+-----------+----+-----+------+-----------+--------+--------+-----+-------+--------------+----------------+
|    AAL|AMERICAN AIRLINES...|ESTRANGEIRA|2000|    1|  KDFW|       SBGR|      30|  247320| 6330| 409.08|          3932|             142|
|    AAL|AMERICAN AIRLINES...|ESTRANGEIRA|2000|    1|  KJFK|       SBGL|      29|  224141| 6119|  418.1|          2338|             131|
|    AAL|AMERICAN AIRLINES...|ESTRANGEIRA|2000|    1|  KJFK|       SBGR|      29|  222256| 6119|  411.5|          2892|             127|
|    AAL|AMERICAN AIRLINES...|ESTRANGEIRA|2000|    1|  KMIA|       SBCF|    null|    null| null|   null|           997|              20|
|    AAL|AMERICAN AIRLINES...|ESTRANGEIRA

In [77]:
estatisticosSelect.where('year == 2000').show()

+-------+--------------------+-----------+----+-----+------+-----------+--------+--------+-----+-------+--------------+----------------+
|company|         companyName|nationality|year|month|origin|destination|takeOffs|distance|seats|   time|paidPassengers|noPaidPassengers|
+-------+--------------------+-----------+----+-----+------+-----------+--------+--------+-----+-------+--------------+----------------+
|    AAL|AMERICAN AIRLINES...|ESTRANGEIRA|2000|    1|  KDFW|       SBGR|      30|  247320| 6330| 409.08|          3932|             142|
|    AAL|AMERICAN AIRLINES...|ESTRANGEIRA|2000|    1|  KJFK|       SBGL|      29|  224141| 6119|  418.1|          2338|             131|
|    AAL|AMERICAN AIRLINES...|ESTRANGEIRA|2000|    1|  KJFK|       SBGR|      29|  222256| 6119|  411.5|          2892|             127|
|    AAL|AMERICAN AIRLINES...|ESTRANGEIRA|2000|    1|  KMIA|       SBCF|    null|    null| null|   null|           997|              20|
|    AAL|AMERICAN AIRLINES...|ESTRANGEIRA

In [78]:
estatisticos2019To2022MCZ = estatisticosSelect.where('year >= 2019').where('year <= 2022').where('origin == "SBMO"')

In [79]:
estatisticos2019To2022MCZ.count()

1447

In [80]:
estatisticosParquet = '/home/danilojpfreitas/Documentos/Data Engineer/Projeto/DataPipeline-TarifasAereas/data/processing/estatisticos'

In [81]:
estatisticos2019To2022MCZ.write.parquet(
    estatisticosParquet,
    mode = 'overwrite'
)

In [82]:
estatisticosParquet = spark.read.parquet(
    estatisticosParquet
)

In [83]:
estatisticosParquet.show()

+-------+--------------------+-----------+----+-----+------+-----------+--------+--------+-----+-------+--------------+----------------+
|company|         companyName|nationality|year|month|origin|destination|takeOffs|distance|seats|   time|paidPassengers|noPaidPassengers|
+-------+--------------------+-----------+----+-----+------+-----------+--------+--------+-----+-------+--------------+----------------+
|    TAM|TAM LINHAS AÉREAS...| BRASILEIRA|2019|   11|  SBMO|       SBBR|      60|   89580|11512| 140.46|          9467|              42|
|    TAM|TAM LINHAS AÉREAS...| BRASILEIRA|2019|   11|  SBMO|       SBGR|     141|  270720|27208| 419.36|         22874|             202|
|    TAM|TAM LINHAS AÉREAS...| BRASILEIRA|2019|   11|  SBMO|       SBPA|       5|   13910|  882|  20.01|           743|               0|
|    TAM|TAM LINHAS AÉREAS...| BRASILEIRA|2019|   11|  SBMO|       SBRP|       1|    1821|  162|   3.27|             0|               0|
|    TAM|TAM LINHAS AÉREAS...| BRASILEIRA

In [84]:
estatisticosParquet.count()

1447

# Análise dos Dados

## Views

In [313]:
price2019Parquet.createOrReplaceTempView("2019View")
price2020Parquet.createOrReplaceTempView("2020View")
price2021Parquet.createOrReplaceTempView("2021View")
price2022Parquet.createOrReplaceTempView("2022View")

In [314]:
spark.sql("SELECT * FROM 2020View").show(5)

+----+-----+-------+------+-----------+------+-----+
|year|month|company|origin|destination| price|seats|
+----+-----+-------+------+-----------+------+-----+
|2020|   11|    AZU|  SBMO|       SBAE|1370.9|    2|
|2020|   11|    AZU|  SBMO|       SBAE|1409.9|    2|
|2020|   11|    AZU|  SBMO|       SBAE| 352.8|    6|
|2020|   11|    AZU|  SBMO|       SBAE|378.05|    1|
|2020|   11|    AZU|  SBMO|       SBAE| 415.9|    2|
+----+-----+-------+------+-----------+------+-----+
only showing top 5 rows



In [315]:
spark.sql("""
        SELECT 
        destination, count(origin)
        FROM
        2019View
        GROUP BY destination
        ORDER BY count(origin) DESC;
        """).show(1000)

+-----------+-------------+
|destination|count(origin)|
+-----------+-------------+
|       SBGR|         8240|
|       SBBR|         4852|
|       SBGL|         3411|
|       SBPA|         3011|
|       SBKP|         2938|
|       SBCF|         2925|
|       SBCT|         2370|
|       SBSP|         2266|
|       SBRJ|         1857|
|       SBCG|         1823|
|       SBFL|         1794|
|       SBSV|         1630|
|       SBGO|         1539|
|       SBCY|         1527|
|       SBRP|         1314|
|       SBEG|         1232|
|       SBLO|         1208|
|       SBVT|         1197|
|       SBSR|         1135|
|       SBRF|         1018|
|       SBPV|         1016|
|       SBBE|          931|
|       SBFZ|          928|
|       SBUL|          861|
|       SBFI|          778|
|       SBNF|          729|
|       SBRB|          703|
|       SBSL|          665|
|       SBTE|          621|
|       SBPJ|          518|
|       SBSG|          513|
|       SBMG|          430|
|       SBJV|       

In [316]:
icao_parquet.createOrReplaceTempView("icaoView")

In [317]:
spark.sql("""
        SELECT 
        *
        FROM
        icaoView;
        """).show(10)

+----+--------------------+--------------------+-----+
|icao|             airport|                city|state|
+----+--------------------+--------------------+-----+
|SBFE|JOÃO DURVAL CARNEIRO|    FEIRA DE SANTANA|   BA|
|SSPB|         PATO BRANCO|         PATO BRANCO|   PR|
|SBRG|          RIO GRANDE|          RIO GRANDE|   RS|
|SBGU|TANCREDO THOMAS D...|          GUARAPUAVA|   PR|
|SSOU|            ARIPUANÃ|            ARIPUANÃ|   MT|
|SBSI|   FAZENDA ITAGUASSU|            MARACAJU|   MS|
|SWSI|PRESIDENTE JOÃO B...|              SINOPE|   MG|
|SBQV|PEDRO OTACÍLIO FI...|VITÓRIA DA CONQUISTA|   BA|
|SBAA|CONCEIÇÃO DO ARAG...|CONCEIÇÃO DO ARAG...|   PA|
|SBAE|       BAURU/AREALVA|               BAURU|   SP|
+----+--------------------+--------------------+-----+
only showing top 10 rows



In [90]:
dist_parquet.createOrReplaceTempView("distView")

In [91]:
spark.sql("""
        SELECT
        *
        FROM
        distView;
        """).show(10)

+----------+-----------+--------+
|      city|distHighway|distLine|
+----------+-----------+--------+
|Rio Branco|       5063|    3513|
|    Macapa|       3545|    2011|
|    Manaus|       5444|    2779|
|  Salvador|        578|     476|
| Fortaleza|        965|     731|
|  Brasilia|       1920|    1487|
|   Vitoria|       1611|    1282|
|   Goiania|       2119|    1658|
|  Sao Luis|       1584|    1236|
|    Cuiaba|       3111|    2304|
+----------+-----------+--------+
only showing top 10 rows



In [92]:
estatisticosParquet.createOrReplaceTempView("estatisticosView")

In [93]:
spark.sql("""
    SELECT
    *
    FROM
    estatisticosView
""").show(10)

+-------+--------------------+-----------+----+-----+------+-----------+--------+--------+-----+------+--------------+----------------+
|company|         companyName|nationality|year|month|origin|destination|takeOffs|distance|seats|  time|paidPassengers|noPaidPassengers|
+-------+--------------------+-----------+----+-----+------+-----------+--------+--------+-----+------+--------------+----------------+
|    TAM|TAM LINHAS AÉREAS...| BRASILEIRA|2019|   11|  SBMO|       SBBR|      60|   89580|11512|140.46|          9467|              42|
|    TAM|TAM LINHAS AÉREAS...| BRASILEIRA|2019|   11|  SBMO|       SBGR|     141|  270720|27208|419.36|         22874|             202|
|    TAM|TAM LINHAS AÉREAS...| BRASILEIRA|2019|   11|  SBMO|       SBPA|       5|   13910|  882| 20.01|           743|               0|
|    TAM|TAM LINHAS AÉREAS...| BRASILEIRA|2019|   11|  SBMO|       SBRP|       1|    1821|  162|  3.27|             0|               0|
|    TAM|TAM LINHAS AÉREAS...| BRASILEIRA|2019| 

### Testes com Estatisticos

In [108]:
spark.sql("""
    SELECT
    *
    FROM
    estatisticosView
    WHERE month == 5 AND year == 2019 AND company == 'AZU'  
""").show(5)

+-------+--------------------+-----------+----+-----+------+-----------+--------+--------+-----+-------+--------------+----------------+
|company|         companyName|nationality|year|month|origin|destination|takeOffs|distance|seats|   time|paidPassengers|noPaidPassengers|
+-------+--------------------+-----------+----+-----+------+-----------+--------+--------+-----+-------+--------------+----------------+
|    AZU|AZUL LINHAS AÉREA...| BRASILEIRA|2019|    5|  SBMO|       SBCF|      12|   17124| 1640|28.0333|          1584|              18|
|    AZU|AZUL LINHAS AÉREA...| BRASILEIRA|2019|    5|  SBMO|       SBKP|      36|   69336| 5463|105.883|          5008|             181|
|    AZU|AZUL LINHAS AÉREA...| BRASILEIRA|2019|    5|  SBMO|       SBRF|     110|   19910| 7810|   90.8|          6683|             139|
|    AZU|AZUL LINHAS AÉREA...| BRASILEIRA|2019|    5|  SBMO|       SBRP|       2|    3642|  236|6.16667|           216|               5|
|    AZU|AZUL LINHAS AÉREA...| BRASILEIRA

In [131]:
spark.sql("""
        SELECT 
        *
        FROM
        2019View
""").show()

+----+-----+-------+------+-----------+------+-----+
|year|month|company|origin|destination| price|seats|
+----+-----+-------+------+-----------+------+-----+
|2019|    5|    AZU|  SBMO|       SBAE| 558.9|    3|
|2019|    5|    AZU|  SBMO|       SBAE| 633.9|    3|
|2019|    5|    AZU|  SBMO|       SBAE| 648.9|    1|
|2019|    5|    AZU|  SBMO|       SBAE| 723.9|    2|
|2019|    5|    AZU|  SBMO|       SBAE| 773.9|    2|
|2019|    5|    AZU|  SBMO|       SBAE| 845.9|    6|
|2019|    5|    AZU|  SBMO|       SBAE|1007.9|    3|
|2019|    5|    AZU|  SBMO|       SBAE|1080.9|    1|
|2019|    5|    AZU|  SBMO|       SBAE|1372.9|    2|
|2019|    5|    AZU|  SBMO|       SBAE|1453.9|    1|
|2019|    5|    AZU|  SBMO|       SBAR| 445.9|    1|
|2019|    5|    AZU|  SBMO|       SBAR| 530.9|    1|
|2019|    5|    AZU|  SBMO|       SBAR| 595.9|    1|
|2019|    5|    AZU|  SBMO|       SBAR| 684.9|    1|
|2019|    5|    AZU|  SBMO|       SBAR| 725.9|    1|
|2019|    5|    AZU|  SBMO|       SBAR| 749.9|

In [140]:
avgPrice2019 = spark.sql("""
        SELECT
        year, month, company, origin, destination, 
        ROUND((SUM(price*seats)/SUM(seats)), 2) AS priceAVG, SUM(seats) AS seats
        FROM
        2019View    
        GROUP BY year, month, company, origin, destination
        ORDER BY month
""")

In [142]:
avgPrice2019.createOrReplaceTempView("avgPrice2019View")

In [144]:
avgPrice2019.printSchema()

root
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- company: string (nullable = true)
 |-- origin: string (nullable = true)
 |-- destination: string (nullable = true)
 |-- priceAVG: double (nullable = true)
 |-- seats: long (nullable = true)



In [145]:
joinAvgPriceEstatisticos2019 = spark.sql("""
        SELECT
        A.year, A.month, A.company, B.companyName, C.city,
        C.state, A.destination, A.priceAVG, A.seats AS seatsAVG, 
        B.takeOffs, B.distance, B.seats AS seatsAll,
        B.time
        FROM
        avgPrice2019View A
        LEFT JOIN
        (SELECT * FROM estatisticosView WHERE year == 2019) B
        ON A.month = B.month AND A.company = B.company AND A.destination == B.destination
        LEFT JOIN
        icaoView C
        ON A.destination = C.icao
""")

In [149]:
joinAvgPriceEstatisticos2019.count()

2296

In [150]:
joinAvgPriceEstatisticos2019.printSchema()

root
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- company: string (nullable = true)
 |-- companyName: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- destination: string (nullable = true)
 |-- priceAVG: double (nullable = true)
 |-- seatsAVG: long (nullable = true)
 |-- takeOffs: integer (nullable = true)
 |-- distance: integer (nullable = true)
 |-- seatsAll: integer (nullable = true)
 |-- time: string (nullable = true)



In [166]:
joinAvgPriceEstatisticos2019.where(test.time.isNotNull()).count()

341

### Tabelas Estatisticos e Price não podem ser unidas! -> Criar Separadas

## Junção das tabelas por ano

### 2019

In [248]:
avgPrice2019 = spark.sql("""
        SELECT
        year, month, company, origin, destination, 
        ROUND((SUM(price*seats)/SUM(seats)), 2) AS priceAVG, SUM(seats) AS seats
        FROM
        2019View    
        GROUP BY year, month, company, origin, destination
        ORDER BY month
""")

In [249]:
avgPrice2019.createOrReplaceTempView("avgPrice2019View")

In [250]:
data2019 = spark.sql("""
        SELECT
        A.year, A.month, A.company, B.companyName, C.city,
        C.state, A.destination, A.priceAVG, A.seats AS seatsAVG, 
        B.takeOffs, B.distance, B.seats AS seatsAll,
        B.time
        FROM
        avgPrice2019View A
        LEFT JOIN
        (SELECT * FROM estatisticosView WHERE year == 2019) B
        ON A.month = B.month AND A.company = B.company AND A.destination == B.destination
        LEFT JOIN
        icaoView C
        ON A.destination = C.icao
        """)

In [251]:
data2019 = data2019.where(data2019.time.isNotNull())

In [252]:
data2019.limit(5).show()

+----+-----+-------+--------------------+--------------+-----+-----------+--------+--------+--------+--------+--------+------+
|year|month|company|         companyName|          city|state|destination|priceAVG|seatsAVG|takeOffs|distance|seatsAll|  time|
+----+-----+-------+--------------------+--------------+-----+-----------+--------+--------+--------+--------+--------+------+
|2019|    5|    TAM|TAM LINHAS AÉREAS...|RIO DE JANEIRO|   RJ|       SBGL|  631.47|     535|       8|   13424|    1760| 19.85|
|2019|    1|    TAM|TAM LINHAS AÉREAS...|RIO DE JANEIRO|   RJ|       SBGL|  532.17|     904|      12|   20136|    2456| 30.04|
|2019|    6|    GLO|GOL LINHAS AÉREAS...|     SÃO PAULO|   SP|       SBSP|  508.06|    1861|      40|   77920|    7296|121.86|
|2019|    6|    GLO|GOL LINHAS AÉREAS...|     SÃO PAULO|   SP|       SBSP|  508.06|    1861|       1|    1948|     138|  3.08|
|2019|    1|    GLO|GOL LINHAS AÉREAS...|     SÃO PAULO|   SP|       SBSP|  537.27|    2670|      38|   74024| 

In [253]:
data2019.count()

341

In [254]:
pathData2019 = "/home/danilojpfreitas/Documentos/Data Engineer/Projeto/DataPipeline-TarifasAereas/data/curated/data/2019"

In [255]:
data2019.write.option("header",True).option("delimiter", ",").mode("overwrite").csv(pathData2019)

In [256]:
curated2019 = spark.read.csv(pathData2019, sep=',', header=True) 

In [257]:
curated2019.limit(10).show()

+----+-----+-------+--------------------+--------------+-----+-----------+--------+--------+--------+--------+--------+------+
|year|month|company|         companyName|          city|state|destination|priceAVG|seatsAVG|takeOffs|distance|seatsAll|  time|
+----+-----+-------+--------------------+--------------+-----+-----------+--------+--------+--------+--------+--------+------+
|2019|    5|    TAM|TAM LINHAS AÉREAS...|RIO DE JANEIRO|   RJ|       SBGL|  631.47|     535|       8|   13424|    1760| 19.85|
|2019|    1|    TAM|TAM LINHAS AÉREAS...|RIO DE JANEIRO|   RJ|       SBGL|  532.17|     904|      12|   20136|    2456| 30.04|
|2019|    6|    GLO|GOL LINHAS AÉREAS...|     SÃO PAULO|   SP|       SBSP|  508.06|    1861|      40|   77920|    7296|121.86|
|2019|    6|    GLO|GOL LINHAS AÉREAS...|     SÃO PAULO|   SP|       SBSP|  508.06|    1861|       1|    1948|     138|  3.08|
|2019|    1|    GLO|GOL LINHAS AÉREAS...|     SÃO PAULO|   SP|       SBSP|  537.27|    2670|      38|   74024| 

In [258]:
curated2019.where(curated2019.city.isNull()).show()

+----+-----+-------+-----------+----+-----+-----------+--------+--------+--------+--------+--------+----+
|year|month|company|companyName|city|state|destination|priceAVG|seatsAVG|takeOffs|distance|seatsAll|time|
+----+-----+-------+-----------+----+-----+-----------+--------+--------+--------+--------+--------+----+
+----+-----+-------+-----------+----+-----+-----------+--------+--------+--------+--------+--------+----+



### 2020

In [259]:
avgPrice2020 = spark.sql("""
        SELECT
        year, month, company, origin, destination, 
        ROUND((SUM(price*seats)/SUM(seats)), 2) AS priceAVG, SUM(seats) AS seats
        FROM
        2020View    
        GROUP BY year, month, company, origin, destination
        ORDER BY month
""")

In [260]:
avgPrice2020.createOrReplaceTempView("avgPrice2020View")

In [261]:
data2020 = spark.sql("""
        SELECT
        A.year, A.month, A.company, B.companyName, C.city,
        C.state, A.destination, A.priceAVG, A.seats AS seatsAVG, 
        B.takeOffs, B.distance, B.seats AS seatsAll,
        B.time
        FROM
        avgPrice2020View A
        LEFT JOIN
        (SELECT * FROM estatisticosView WHERE year == 2020) B
        ON A.month = B.month AND A.company = B.company AND A.destination == B.destination
        LEFT JOIN
        icaoView C
        ON A.destination = C.icao
        """)

In [262]:
data2020 = data2020.where(data2020.time.isNotNull())

In [263]:
data2020.limit(5).show()

+----+-----+-------+--------------------+--------------+-----+-----------+--------+--------+--------+--------+--------+-------+
|year|month|company|         companyName|          city|state|destination|priceAVG|seatsAVG|takeOffs|distance|seatsAll|   time|
+----+-----+-------+--------------------+--------------+-----+-----------+--------+--------+--------+--------+--------+-------+
|2020|   10|    GLO|GOL LINHAS AÉREAS...|      SALVADOR|   BA|       SBSV|  537.64|     241|       1|     468|     186|1.16667|
|2020|   11|    TAM|TAM LINHAS AÉREAS...|RIO DE JANEIRO|   RJ|       SBGL|  692.92|     157|       1|    1678|     214|3.66667|
|2020|   10|    TAM|TAM LINHAS AÉREAS...|     GUARULHOS|   SP|       SBGR|  625.82|    4580|      79|  151680|   14918|235.583|
|2020|   11|    GLO|GOL LINHAS AÉREAS...|RIO DE JANEIRO|   RJ|       SBGL|  441.24|    3372|      30|   50340|    5499|83.6167|
|2020|   10|    GLO|GOL LINHAS AÉREAS...|RIO DE JANEIRO|   RJ|       SBGL|  712.89|     941|      31|   

In [264]:
data2020.count()

186

In [265]:
pathData2020 = "/home/danilojpfreitas/Documentos/Data Engineer/Projeto/DataPipeline-TarifasAereas/data/curated/data/2020"

In [266]:
data2020.write.option("header",True).option("delimiter", ",").mode("overwrite").csv(pathData2020)

In [267]:
curated2020 = spark.read.csv(pathData2020, sep=',', header=True) 

In [268]:
curated2020.limit(10).show()

+----+-----+-------+--------------------+--------------+-----+-----------+--------+--------+--------+--------+--------+-------+
|year|month|company|         companyName|          city|state|destination|priceAVG|seatsAVG|takeOffs|distance|seatsAll|   time|
+----+-----+-------+--------------------+--------------+-----+-----------+--------+--------+--------+--------+--------+-------+
|2020|   10|    GLO|GOL LINHAS AÉREAS...|      SALVADOR|   BA|       SBSV|  537.64|     241|       1|     468|     186|1.16667|
|2020|   11|    TAM|TAM LINHAS AÉREAS...|RIO DE JANEIRO|   RJ|       SBGL|  692.92|     157|       1|    1678|     214|3.66667|
|2020|   10|    TAM|TAM LINHAS AÉREAS...|     GUARULHOS|   SP|       SBGR|  625.82|    4580|      79|  151680|   14918|235.583|
|2020|   11|    GLO|GOL LINHAS AÉREAS...|RIO DE JANEIRO|   RJ|       SBGL|  441.24|    3372|      30|   50340|    5499|83.6167|
|2020|   10|    GLO|GOL LINHAS AÉREAS...|RIO DE JANEIRO|   RJ|       SBGL|  712.89|     941|      31|   

In [269]:
curated2020.where(curated2020.city.isNull()).show()

+----+-----+-------+-----------+----+-----+-----------+--------+--------+--------+--------+--------+----+
|year|month|company|companyName|city|state|destination|priceAVG|seatsAVG|takeOffs|distance|seatsAll|time|
+----+-----+-------+-----------+----+-----+-----------+--------+--------+--------+--------+--------+----+
+----+-----+-------+-----------+----+-----+-----------+--------+--------+--------+--------+--------+----+



### 2021

In [270]:
avgPrice2021 = spark.sql("""
        SELECT
        year, month, company, origin, destination, 
        ROUND((SUM(price*seats)/SUM(seats)), 2) AS priceAVG, SUM(seats) AS seats
        FROM
        2021View    
        GROUP BY year, month, company, origin, destination
        ORDER BY month
""")

In [271]:
avgPrice2021.createOrReplaceTempView("avgPrice2021View")

In [272]:
data2021 = spark.sql("""
        SELECT
        A.year, A.month, A.company, B.companyName, C.city,
        C.state, A.destination, A.priceAVG, A.seats AS seatsAVG, 
        B.takeOffs, B.distance, B.seats AS seatsAll,
        B.time
        FROM
        avgPrice2021View A
        LEFT JOIN
        (SELECT * FROM estatisticosView WHERE year == 2021) B
        ON A.month = B.month AND A.company = B.company AND A.destination == B.destination
        LEFT JOIN
        icaoView C
        ON A.destination = C.icao
        """)

In [273]:
data2021 = data2021.where(data2021.time.isNotNull())

In [274]:
data2021.limit(5).show()

+----+-----+-------+--------------------+------------+-----+-----------+--------+--------+--------+--------+--------+-------+
|year|month|company|         companyName|        city|state|destination|priceAVG|seatsAVG|takeOffs|distance|seatsAll|   time|
+----+-----+-------+--------------------+------------+-----+-----------+--------+--------+--------+--------+--------+-------+
|2021|   11|    TAM|TAM LINHAS AÉREAS...|   FORTALEZA|   CE|       SBFZ|  415.28|     807|      21|   14826|    4392|28.7667|
|2021|   11|    TAM|TAM LINHAS AÉREAS...|   FORTALEZA|   CE|       SBFZ|  415.28|     807|       1|     706|     216|    1.3|
|2021|   11|    AZU|AZUL LINHAS AÉREA...|PORTO ALEGRE|   RS|       SBPA|  715.74|     861|       4|   11128|     575|   16.8|
|2021|   11|    AZU|AZUL LINHAS AÉREA...|     CONFINS|   MG|       SBCF|  747.05|     596|       4|    5708|     472|9.08333|
|2021|   11|    AZU|AZUL LINHAS AÉREA...|     CONFINS|   MG|       SBCF|  747.05|     596|       3|    4281|     354| 

In [275]:
data2021.count()

234

In [276]:
pathData2021 = "/home/danilojpfreitas/Documentos/Data Engineer/Projeto/DataPipeline-TarifasAereas/data/curated/data/2021"

In [277]:
data2021.write.option("header",True).option("delimiter", ",").mode("overwrite").csv(pathData2021)

In [278]:
curated2021 = spark.read.csv(pathData2021, sep=',', header=True) 

In [279]:
curated2021.limit(10).show()

+----+-----+-------+--------------------+--------------------+-----+-----------+--------+--------+--------+--------+--------+-------+
|year|month|company|         companyName|                city|state|destination|priceAVG|seatsAVG|takeOffs|distance|seatsAll|   time|
+----+-----+-------+--------------------+--------------------+-----+-----------+--------+--------+--------+--------+--------+-------+
|2021|   11|    TAM|TAM LINHAS AÉREAS...|           FORTALEZA|   CE|       SBFZ|  415.28|     807|      21|   14826|    4392|28.7667|
|2021|   11|    TAM|TAM LINHAS AÉREAS...|           FORTALEZA|   CE|       SBFZ|  415.28|     807|       1|     706|     216|    1.3|
|2021|   11|    AZU|AZUL LINHAS AÉREA...|        PORTO ALEGRE|   RS|       SBPA|  715.74|     861|       4|   11128|     575|   16.8|
|2021|   11|    AZU|AZUL LINHAS AÉREA...|             CONFINS|   MG|       SBCF|  747.05|     596|       4|    5708|     472|9.08333|
|2021|   11|    AZU|AZUL LINHAS AÉREA...|             CONFINS|

In [280]:
curated2021.where(curated2021.city.isNull()).show()

+----+-----+-------+-----------+----+-----+-----------+--------+--------+--------+--------+--------+----+
|year|month|company|companyName|city|state|destination|priceAVG|seatsAVG|takeOffs|distance|seatsAll|time|
+----+-----+-------+-----------+----+-----+-----------+--------+--------+--------+--------+--------+----+
+----+-----+-------+-----------+----+-----+-----------+--------+--------+--------+--------+--------+----+



### 2022

In [318]:
avgPrice2022 = spark.sql("""
        SELECT
        year, month, company, origin, destination, 
        ROUND((SUM(price*seats)/SUM(seats)), 2) AS priceAVG, SUM(seats) AS seats
        FROM
        2022View    
        GROUP BY year, month, company, origin, destination
        ORDER BY month
""")

In [319]:
avgPrice2022.show()

+----+-----+-------+------+-----------+--------+-----+
|year|month|company|origin|destination|priceAVG|seats|
+----+-----+-------+------+-----------+--------+-----+
|2022|    1|    GLO|  SBMO|       SBGL|  652.75| 2769|
|2022|    1|    GLO|  SBMO|       SBNF| 1932.81|   22|
|2022|    1|    TAM|  SBMO|       SBCH|  539.67|    6|
|2022|    1|    GLO|  SBMO|       SBFZ|  1837.4|    6|
|2022|    1|    AZU|  SBMO|       SBNF|  927.15|   19|
|2022|    1|    GLO|  SBMO|       SBCH| 1500.04|   17|
|2022|    1|    GLO|  SBMO|       SBTE|  2259.0|    3|
|2022|    1|    AZU|  SBMO|       SBPA|  711.43|  420|
|2022|    1|    AZU|  SBMO|       SBPJ|  633.31|   42|
|2022|    1|    AZU|  SBMO|       SBFZ|  414.43|   36|
|2022|    1|    TAM|  SBMO|       SBCY| 1000.91|  157|
|2022|    1|    AZU|  SBMO|       SBCG|  819.33|   95|
|2022|    1|    GLO|  SBMO|       SBJP|  1973.4|    2|
|2022|    1|    GLO|  SBMO|       SBEG| 1216.19|   26|
|2022|    1|    AZU|  SBMO|       SBAR|  1775.4|    2|
|2022|    

In [320]:
avgPrice2022.createOrReplaceTempView("avgPrice2022View")

In [321]:
data2022 = spark.sql("""
        SELECT
        A.year, A.month, A.company, B.companyName, C.city,
        C.state, A.destination, A.priceAVG, A.seats AS seatsAVG, 
        B.takeOffs, B.distance, B.seats AS seatsAll,
        B.time
        FROM
        avgPrice2022View A
        LEFT JOIN
        (SELECT * FROM estatisticosView WHERE year == 2022) B
        ON A.month = B.month AND A.company = B.company AND A.destination == B.destination
        LEFT JOIN
        icaoView C
        ON A.destination = C.icao
        """)

In [322]:
data2022 = data2022.where(data2022.time.isNotNull())

In [323]:
data2022.limit(5).show()

+----+-----+-------+--------------------+--------------+-----+-----------+--------+--------+--------+--------+--------+-------+
|year|month|company|         companyName|          city|state|destination|priceAVG|seatsAVG|takeOffs|distance|seatsAll|   time|
+----+-----+-------+--------------------+--------------+-----+-----------+--------+--------+--------+--------+--------+-------+
|2022|   12|    GLO|GOL LINHAS AÉREAS...|    UBERLÂNDIA|   MG|       SBUL|   683.6|     248|       4|    6784|     696|   10.9|
|2022|   12|    GLO|GOL LINHAS AÉREAS...|      SALVADOR|   BA|       SBSV|  590.95|     433|      31|   14508|    5646|36.3167|
|2022|   10|    AZU|AZUL LINHAS AÉREA...|RIBEIRÃO PRETO|   SP|       SBRP| 1188.26|     165|      10|   18210|    1731|27.6667|
|2022|   10|    AZU|AZUL LINHAS AÉREA...|    UBERLÂNDIA|   MG|       SBUL|  1004.7|     141|       5|    8480|     861|12.9167|
|2022|   12|    AZU|AZUL LINHAS AÉREA...|      CAMPINAS|   SP|       SBKP| 1101.02|     480|      43|   

In [324]:
data2022.count()

361

In [325]:
pathData2022 = "/home/danilojpfreitas/Documentos/Data Engineer/Projeto/DataPipeline-TarifasAereas/data/curated/data/2022"

In [326]:
data2022.write.option("header",True).option("delimiter", ",").mode("overwrite").csv(pathData2022)

In [327]:
curated2022 = spark.read.csv(pathData2022, sep=',', header=True) 

In [328]:
curated2022.limit(10).show()

+----+-----+-------+--------------------+--------------+-----+-----------+--------+--------+--------+--------+--------+-------+
|year|month|company|         companyName|          city|state|destination|priceAVG|seatsAVG|takeOffs|distance|seatsAll|   time|
+----+-----+-------+--------------------+--------------+-----+-----------+--------+--------+--------+--------+--------+-------+
|2022|   12|    GLO|GOL LINHAS AÉREAS...|    UBERLÂNDIA|   MG|       SBUL|   683.6|     248|       4|    6784|     696|   10.9|
|2022|   12|    GLO|GOL LINHAS AÉREAS...|      SALVADOR|   BA|       SBSV|  590.95|     433|      31|   14508|    5646|36.3167|
|2022|   10|    AZU|AZUL LINHAS AÉREA...|RIBEIRÃO PRETO|   SP|       SBRP| 1188.26|     165|      10|   18210|    1731|27.6667|
|2022|   10|    AZU|AZUL LINHAS AÉREA...|    UBERLÂNDIA|   MG|       SBUL|  1004.7|     141|       5|    8480|     861|12.9167|
|2022|   12|    AZU|AZUL LINHAS AÉREA...|      CAMPINAS|   SP|       SBKP| 1101.02|     480|      43|   

In [329]:
curated2022.where(curated2022.city.isNull()).show()

+----+-----+-------+-----------+----+-----+-----------+--------+--------+--------+--------+--------+----+
|year|month|company|companyName|city|state|destination|priceAVG|seatsAVG|takeOffs|distance|seatsAll|time|
+----+-----+-------+-----------+----+-----+-----------+--------+--------+--------+--------+--------+----+
+----+-----+-------+-----------+----+-----+-----------+--------+--------+--------+--------+--------+----+

